In [ ]:
# prompt: /content/df.csv read

import pandas as pd

df = pd.read_csv('/content/DS-ALS-REDSEA-CLEANED.csv')
df

,Timestamp,Speed,Course,Latitude,Longitude,Vessel
0,2024-07-01 00:17:08,0.00,215.00,21.47355,39.16373,NewVessel_217
1,2024-07-01 00:17:08,0.00,215.00,21.47355,39.16373,NewVessel_13
2,2024-07-01 00:17:08,0.00,215.00,21.47355,39.16373,MarineTraffic_Vessel_positions_Export_2024-07-...
3,2024-07-01 00:17:08,0.00,215.00,21.47355,39.16373,NewVessel_299
4,2024-07-01 00:17:08,0.00,215.00,21.47355,39.16373,NewVessel_259
...,...,...,...,...,...,...
140998,2024-11-22 23:59 UTC,14.81,337.01,30.11000,33.15000,AQUA LADY_9288435_249807000
140999,2024-11-22 23:59 UTC,14.80,337.00,29.56003,32.57593,SCIROCCO_9407835_636014043
141000,2024-11-22 23:59 UTC,11.43,312.02,28.53000,34.26000,FORTUNE STAR_9436721_538009251
141001,2024-11-22 23:59 UTC,12.12,144.02,26.59000,35.70000,NAXOS_9336426_352002674


In [ ]:
Timestamp=df['Timestamp']
Speed=df['Speed']
Course=df['Course']
Latitude=df['Latitude']
Longitude=df['Longitude']
Vessel=df['Vessel']
new_df=pd.DataFrame({'Timestamp':Timestamp,'Speed':Speed,'Course':Course,'Latitude':Latitude,'Longitude':Longitude,'Vessel':Vessel})

#  REMOVE  VALUE masked     IN TABLE
df_cleaned = new_df.dropna()

df_cleaned = df_cleaned.replace('masked', pd.NA)
df_cleaned = df_cleaned.dropna()
df_cleaned['Latitude'] = df_cleaned['Latitude'].astype(float)
df_cleaned['Longitude'] = df_cleaned['Longitude'].astype(float)
df_cleaned['Speed'] = df_cleaned['Speed'].astype(float)
df_cleaned['Course']=df_cleaned['Course'].astype(float)

df_cleaned.sort_values(by=['Timestamp'], inplace=True)
df_cleaned.reset_index(drop=True, inplace=True)

import torch
vs=df_cleaned['Vessel'].unique()
def  get_labels(num):
     if num <100:
       return 0
     elif num <200:
       return 1
     else:
       return 2
data=[]
labels=[]
features=['Latitude','Longitude','Speed','Course']
for v in vs:
  v1=df_cleaned[df_cleaned['Vessel']==v]

  data.append(torch.tensor(v1[features].values)/360)
  labels.append([get_labels(len(v1))]*4)
data=data*1
labels=labels*1
labels=torch.tensor(labels)

In [ ]:
import numpy as np
import pickle
import os

# تعريف عدد الخانات لكل من المتغيرات
LAT_BINS = 200
LON_BINS = 300
SOG_BINS = 30
COG_BINS = 72

# دالة لتحويل البيانات إلى مصفوفة كثيفة
def sparse_AIS_to_dense(msgs_, num_timesteps, mmsis):
    def create_dense_vect(msg, lat_bins=LAT_BINS, lon_bins=LON_BINS, sog_bins=SOG_BINS, cog_bins=COG_BINS):
        lat, lon, sog, cog = msg[0], msg[1], msg[2], msg[3]
        data_dim = lat_bins + lon_bins + sog_bins + cog_bins
        dense_vect = np.zeros(data_dim)
        dense_vect[int(lat * lat_bins)] = 1.0
        dense_vect[int(lon * lon_bins) + lat_bins] = 1.0
        dense_vect[int(sog * sog_bins) + lat_bins + lon_bins] = 1.0
        dense_vect[int(cog * cog_bins) + lat_bins + lon_bins + sog_bins] = 1.0
        return dense_vect

    dense_msgs = []
    for msg in msgs_:
        dense_msgs.append(create_dense_vect(msg))
    dense_msgs = np.array(dense_msgs)
    return dense_msgs, num_timesteps, mmsis

# دالة لحساب المتوسط من قائمة المسارات
def calculate_mean_from_routes(routes):
    data_dim = LAT_BINS + LON_BINS + SOG_BINS + COG_BINS

    # تهيئة المتغيرات لحساب المتوسط
    sum_all = np.zeros((data_dim,))
    total_ais_msg = 0

    # حساب المتوسط
    for count, route in enumerate(routes):
        print(f"Processing route: {count + 1}/{len(routes)}")

        # معالجة البيانات
        current_sparse_matrix, _, _ = sparse_AIS_to_dense(route, 0, 0)

        # جمع البيانات
        sum_all += np.sum(current_sparse_matrix, axis=0)
        total_ais_msg += len(current_sparse_matrix)

    # حساب المتوسط النهائي
    mean = sum_all / total_ais_msg
    return mean

# مثال على كيفية استخدام الدالة مع قائمة من المسارات
if __name__ == "__main__":
    # استبدل هذا بقائمة من المسارات الخاصة بك
    routes = data

    mean = calculate_mean_from_routes(routes)

    # حفظ المتوسط في ملف

    print("Mean calculated and saved successfully.")


Processing route: 1/800
Processing route: 2/800
Processing route: 3/800
Processing route: 4/800
Processing route: 5/800
Processing route: 6/800
Processing route: 7/800
Processing route: 8/800
Processing route: 9/800
Processing route: 10/800
Processing route: 11/800
Processing route: 12/800
Processing route: 13/800
Processing route: 14/800
Processing route: 15/800
Processing route: 16/800
Processing route: 17/800
Processing route: 18/800
Processing route: 19/800
Processing route: 20/800
Processing route: 21/800
Processing route: 22/800
Processing route: 23/800
Processing route: 24/800
Processing route: 25/800
Processing route: 26/800
Processing route: 27/800
Processing route: 28/800
Processing route: 29/800
Processing route: 30/800
Processing route: 31/800
Processing route: 32/800
Processing route: 33/800
Processing route: 34/800
Processing route: 35/800
Processing route: 36/800
Processing route: 37/800
Processing route: 38/800
Processing route: 39/800
Processing route: 40/800
Processin

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import concurrent.futures

LAT, LON, SOG, COG, HEADING, TIMESTAMP = list(range(6))

class AISDataset(Dataset):
    def __init__(self, tracks, lat_bins, lon_bins, sog_bins, cog_bins,mean,divm=360):
        self.tracks = tracks  # قائمة من المسارات
        self.lat_bins = lat_bins
        self.lon_bins = lon_bins
        self.sog_bins = sog_bins
        self.cog_bins = cog_bins
        self.total_bins = lat_bins + lon_bins + sog_bins + cog_bins  # إجمالي عدد الخانات

        # Normalize the mean values for inputs.
        self.mean = mean#self.calculate_mean()
        self.dataFourHot=[]
        self.dataTrackdFourHot=[]
        self.processdata(divm)


    def processdata(self,divm=360):
       outdata=self.get_four_hot_all_data(divm=divm)
       self.dataTrackdFourHot=outdata
       print('finsh get_four_hot_all_data',len(outdata))
       self.dataFourHot=[]
       for  i in range(len(outdata)):
         for j in range(len(outdata[i])):
           self.dataFourHot.append(outdata[i][j,:])

       print('finsh processdata')






    def get_four_hot_all_data(self, divm=360):
      results = []
      for i in range(len(self.tracks)):
              results.append(self.sparse_AIS_to_dense(self.tracks[i]))






      return results

    def sparse_AIS_to_dense_all(self,msgs_,divm=360):
        msgs_=msgs_/divm
        def create_dense_vect(msg, lat_bins=LAT_BINS, lon_bins=LON_BINS, sog_bins=SOG_BINS, cog_bins=COG_BINS):
            lat, lon, sog, cog = msg[0], msg[1], msg[2], msg[3]
            data_dim = lat_bins + lon_bins + sog_bins + cog_bins
            dense_vect = np.zeros(data_dim)
            dense_vect[int(lat * lat_bins)] = 1.0
            dense_vect[int(lon * lon_bins) + lat_bins] = 1.0
            dense_vect[int(sog * sog_bins) + lat_bins + lon_bins] = 1.0
            dense_vect[int(cog * cog_bins) + lat_bins + lon_bins + sog_bins] = 1.0
            return dense_vect

        # التعامل مع القيم 1 في الرسالة
        msgs_[msgs_ == 1] = 0.99999

        dense_msgs = []
        for msg in msgs_:
            dense_msgs.append(create_dense_vect(msg))

        dense_msgs = np.array(dense_msgs)
        return dense_msgs

    def sparse_AIS_to_dense_ones(self, msg):
        dense_vect = np.zeros(self.total_bins)
        dense_vect[int(msg[LAT] * self.lat_bins)] = 1.0
        dense_vect[int(msg[LON] * self.lon_bins) + self.lat_bins] = 1.0
        dense_vect[int(msg[SOG] * self.sog_bins) + self.lat_bins + self.lon_bins] = 1.0
        dense_vect[int(msg[COG] * self.cog_bins) + self.lat_bins + self.lon_bins + self.sog_bins] = 1.0
        return dense_vect
    def sparse_AIS_to_dense(self, msg):
        dense_vect = np.zeros((len(msg),self.total_bins))
        dense_vect[:,np.int16(msg[:,LAT] * self.lat_bins)] = 1.0
        dense_vect[:,np.int16(msg[:,LON] * self.lon_bins) + self.lat_bins] = 1.0
        dense_vect[:,np.int16(msg[:,SOG] * self.sog_bins) + self.lat_bins + self.lon_bins] = 1.0
        dense_vect[:,np.int16(msg[:,COG] * self.cog_bins) + self.lat_bins + self.lon_bins + self.sog_bins] = 1.0
        return dense_vect

    def __len__(self):
        return len(self.dataTrackdFourHot)  # عدد الرسائل في جميع المسارات

    def __getitem__(self, idx):
        if idx >= len(self):
          idx=-1


        inputs =self.dataTrackdFourHot[idx]-self.mean


        # Normalize inputs by subtracting mean


        return torch.FloatTensor(inputs)
def create_dataloader(tracks, batch_size, lat_bins, lon_bins, sog_bins, cog_bins, shuffle=True):
    dataset = AISDataset(tracks, lat_bins, lon_bins, sog_bins, cog_bins)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

# مثال على كيفية استخدام الدالة لإنشاء DataLoader
# استبدل هذا بمساراتك الفعلية

import numpy as np

# تعريف عدد الخانات لكل من المتغيرات
LAT_BINS = 200
LON_BINS = 300
SOG_BINS = 30
COG_BINS = 72


batch_size = 32
lat_bins = 200
lon_bins = 300
sog_bins = 30
cog_bins = 72
# data=[  [
#         [0.1, 0.2, 0.3, 0.4],
#         [0.15, 0.25, 0.35, 0.45],
#         # أضف المزيد من الرسائل حسب الحاجة
#     ]*10]*100

# تحديد قيم size, mean, و total_bins
dataset = AISDataset(data, lat_bins, lon_bins, sog_bins, cog_bins,mean)

print(f"Size of dataset: {len(dataset)}")

print(f"Total bins: {dataset.total_bins}")


finsh get_four_hot_all_data 800
finsh processdata
Size of dataset: 800
Total bins: 602


In [ ]:
import math
import torch
from torch import nn, Tensor
from torch.nn.functional import softplus
from torch.distributions import Distribution
from torch.distributions import Normal
import numpy as np

class VRNN(nn.Module):

    def __init__(self, input_shape, latent_shape, mean_logits, mean_, splits, len_data, gamma, bn_switch):
        super(VRNN, self).__init__()
        self.input_shape = input_shape
        self.latent_shape = latent_shape
        self.mean_logits = mean_logits
        self.mean_ = mean_
        self.splits = splits
        self.len_data = len_data
        self.gamma = gamma
        self.bn_switch = bn_switch

        self.phi_x = nn.Sequential(nn.Linear(self.input_shape, self.latent_shape),
                                   nn.ReLU())

        self.phi_z = nn.Sequential(nn.Linear(self.latent_shape, self.latent_shape),
                                   nn.ReLU())

        self.prior = nn.Sequential(nn.Linear(self.latent_shape, 2*self.latent_shape),
                                   nn.ReLU())

        self.encoder = nn.Sequential(nn.Linear(self.latent_shape + self.latent_shape, 2*self.latent_shape),
                                     nn.ReLU())

        self.decoder = nn.Sequential(nn.Linear(self.latent_shape + self.latent_shape, 2*self.input_shape),
                                     nn.ReLU())

        self.rnn = nn.LSTM(self.latent_shape + self.latent_shape, self.latent_shape, batch_first=True)

        self.register_buffer('out', torch.zeros(1, self.latent_shape))
        self.register_buffer('h', torch.zeros(1, self.latent_shape))
        self.register_buffer('c', torch.zeros(1, 1, self.latent_shape))

        if self.bn_switch:
            self.bn = nn.BatchNorm1d(self.latent_shape)
            self.bn.weight.requires_grad = False
            self.bn.weight.fill_(self.gamma)

    def _prior(self, h):
        hidden = self.prior(h)
        mu, log_sigma = hidden.chunk(2, dim=-1)
        return ReparameterizedDiagonalGaussian(mu, log_sigma)

    def posterior(self, hidden, x, prior_mu):
        encoder_input = torch.cat([hidden, x], dim=1)
        hidden = self.encoder(encoder_input)
        mu, log_sigma = hidden.chunk(2, dim=-1)
        mu = mu + prior_mu
        if self.bn_switch:
            mu = self.bn(mu)
        return ReparameterizedDiagonalGaussian(mu, log_sigma)

    def generative(self, z_enc, h):
        decoder_output = self.decoder(torch.cat([z_enc, h], dim=1))
        mu, log_sigma = decoder_output.chunk(2, dim=-1)
        sigma = softplus(log_sigma)  # Ensure sigma is positive
        return Normal(mu, sigma)

    def calculate_anomaly_score(self, mu, sigma, x):
        # Calculate the z-score
        z_scores = (x - mu) / sigma
        # Anomaly score is defined as the absolute z-score
        return torch.abs(z_scores)

    def calc_mi(self, inputs):

        batch_size = inputs.size(0)

        out = self.out.expand(batch_size, *self.out.shape[1:]).contiguous()
        h = self.h.expand(1, batch_size, self.c.shape[-1]).contiguous()
        c = self.c.expand(1, batch_size, self.c.shape[-1]).contiguous()

        neg_entropy = 0
        log_qz = 0

        for t in range(inputs.size(1)):
            x = inputs[:, t, :]
            x_hat = self.phi_x(x)

            # Create prior distribution
            pz = self._prior(out)

            # Create approximate posterior
            qz = self.posterior(out, x_hat, prior_mu=pz.mu)

            mu = qz.mu
            logsigma = torch.log(qz.sigma)

            z = qz.rsample()
            z_hat = self.phi_z(z)

            rnn_input = torch.cat([x_hat, z_hat], dim=1)
            rnn_input = rnn_input.unsqueeze(1)
            out, (h, c) = self.rnn(rnn_input, (h, c))
            out = out.squeeze()

            neg_entropy += (-0.5 * self.latent_shape * math.log(2 * math.pi) - 0.5 * (1 + 2 * logsigma).sum(-1)).mean()

            var = logsigma.exp()**2

            z = z.unsqueeze(1)
            mu = mu.unsqueeze(0)
            logsigma = logsigma.unsqueeze(0)

            dev = z - mu

            log_density = -0.5 * ((dev ** 2 ) / var).sum(dim=-1) - 0.5 * (self.latent_shape * math.log(2 * math.pi) + (2*logsigma).sum(dim=-1))

            log_qz1 = torch.logsumexp(log_density, dim=1) - math.log(batch_size)
            log_qz += log_qz1.mean(-1)

        mi = (neg_entropy / inputs.size(1)) - (log_qz / inputs.size(1))

        return mi

    def forward(self, inputs, beta):
        batch_size = inputs.size(0)

        out = self.out.expand(batch_size, *self.out.shape[1:]).contiguous()
        h = self.h.expand(1, batch_size, self.c.shape[-1]).contiguous()
        c = self.c.expand(1, batch_size, self.c.shape[-1]).contiguous()

        acc_loss = 0
        loss_list = []
        kl_list = []
        log_px_list = []
        mu_prior = []
        mu_post = []
        noise_levels = []
        accuracy_list = []

        for t in range(inputs.size(1)):
            x = inputs[:, t, :]


            # Embed input
            x_hat = self.phi_x(x)

            # Create prior distribution
            pz = self._prior(out)

            # Create approximate posterior
            qz = self.posterior(out, x_hat, prior_mu=pz.mu)

            # Sample and embed z from posterior
            z = qz.rsample()
            z_hat = self.phi_z(z)

            # Decode z_hat
            px = self.generative(z_hat, out)

            # Update h from LSTM
            rnn_input = torch.cat([x_hat, z_hat], dim=1)
            rnn_input = rnn_input.unsqueeze(1)
            out, (h, c) = self.rnn(rnn_input, (h, c))
            out = out.squeeze()

            # Calculate loss
            log_px = px.log_prob(x).sum(dim=1)
            log_pz = pz.log_prob(z).sum(dim=1)
            log_qz = qz.log_prob(z).sum(dim=1)

            kl = log_qz - log_pz
            elbo_beta = log_px - beta * kl

            acc_loss += elbo_beta
            loss_list.append(-elbo_beta)
            kl_list.append(kl)
            mu_prior.append(pz.mu.sum(dim=1))
            mu_post.append(qz.mu.sum(dim=1))

            # Calculate noise level as the standard deviation of the predicted output
            noise_level = px.stddev.sum(dim=1)
            noise_levels.append(noise_level)

            # Calculate anomaly score
            anomaly_score = self.calculate_anomaly_score(px.mean, px.stddev, x)
            accuracy_list.append(anomaly_score)

        loss = torch.mean(acc_loss / inputs.size(1))

        with torch.no_grad():
            diagnostics = {
                'loss_list': torch.stack(loss_list).cpu().numpy(),
                # 'log_px': torch.stack(log_px_list).cpu().numpy(),
                'kl': torch.stack(kl_list).cpu().numpy(),
                "mu_prior": torch.stack(mu_prior).cpu().numpy(),
                "mu_post": torch.stack(mu_post).cpu().numpy(),
                "noise_levels": torch.stack(noise_levels).cpu().numpy(),
                "accuracy": torch.stack(accuracy_list).cpu().numpy(),
            }

        return -loss, diagnostics

class ReparameterizedDiagonalGaussian(Distribution):
    def __init__(self, mu: Tensor, log_sigma: Tensor):
        assert mu.shape == log_sigma.shape, f"Tensors `mu` : {mu.shape} and ` log_sigma` : {log_sigma.shape} must be of the same shape"
        self.mu = mu
        self.sigma = log_sigma.exp()

    def sample_epsilon(self) -> Tensor:
        return torch.empty_like(self.mu).normal_()

    def sample(self) -> Tensor:
        with torch.no_grad():
            return self.rsample()

    def rsample(self) -> Tensor:
        return self.mu + self.sigma * self.sample_epsilon()

    def log_prob(self, z: Tensor) -> Tensor:
        dist = Normal(self.mu, self.sigma)
        return dist.log_prob(z)

In [ ]:
!pip install wandb


In [ ]:
import wandb
wandb.login(key="dd5a000648e992f5fd8fb3efad3966fe8b8b671f")  # Replace with your API key
# Initialize WandB
#dd5a000648e992f5fd8fb3efad3966fe8b8b671f

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mhmamyr743 (mhmamyr743-wew). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.optim import Adam
from torch.nn.utils.rnn import pad_sequence
import wandb

wandb.init(project='myproject')

def padded_sequences_batch(batch):

    sequences = []

    for sequence in batch:  # Iterate over each item in the batch
        sequences.append(sequence)

    # Pad the sequences
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0.0)
    padded_sequences = padded_sequences.float()


    return padded_sequences

input_shape = 602
latent_shape = 100
mean_logits = 0
mean_ = 0
splits = 1
len_data = 100
gamma = 0.1
bn_switch = True
beta = 1.0
batch_size = 32



train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True,collate_fn=padded_sequences_batch)
val_loader = DataLoader(val_dataset, batch_size=batch_size,collate_fn=padded_sequences_batch)
test_loader = DataLoader(test_dataset, batch_size=batch_size,collate_fn=padded_sequences_batch)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VRNN(input_shape, latent_shape, mean_logits, mean_, splits, len_data, gamma, bn_switch).to(device)
optimizer = Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5000, gamma=0.5)
lst_lossall=[]
lst_klall=[]
lst_anmly=[]
lst_muall=[]
lst_anum_test=[]
lst_mu_test=[]
lst_kl_test=[]
lst_loss_test=[]
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch in train_loader:

        inputs= batch
        inputs=inputs.to(device)
        optimizer.zero_grad()
        loss, diagnostics = model(inputs , beta)

        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    lst_lossall.append(avg_train_loss)
    kl=diagnostics['kl'].mean()
    lst_klall.append(diagnostics['kl'].mean())
    lst_anmly.append(diagnostics['accuracy'].mean())

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs= batch
            inputs=inputs.to(device)
            loss, diagnostics = model(inputs , beta)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    wandb.log({"Train Loss": avg_train_loss,
               "Validation Loss": avg_val_loss,
               "anmaly": diagnostics['accuracy'].mean(),
              "kl": diagnostics['kl'].mean()
    })



    test_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            inputs= batch
            inputs=inputs.to(device)

            loss, diagnostics = model(inputs , beta)
            test_loss += loss.item()

    avg_test_loss = test_loss / len(test_loader)
    lst_loss_test.append(avg_test_loss)
    lst_kl_test.append(diagnostics['kl'].mean())
    lst_anum_test.append(diagnostics['accuracy'].mean())

    wandb.log({"Test Loss": avg_test_loss})
    wandb.log({"Test kl": diagnostics['kl'].mean()})
    wandb.log({"Test anmaly": diagnostics['accuracy'].mean()})

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, kl :{kl}, Validation Loss: {avg_val_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

    scheduler.step()

wandb.finish()


Epoch 1/200, Train Loss: 378.3473, kl :0.6641979217529297, Validation Loss: 371.8550, Test Loss: 370.8964
Epoch 2/200, Train Loss: 362.2416, kl :0.6346280574798584, Validation Loss: 358.0604, Test Loss: 357.1312
Epoch 3/200, Train Loss: 351.4659, kl :0.4980340301990509, Validation Loss: 349.0939, Test Loss: 348.2196
Epoch 4/200, Train Loss: 345.2932, kl :0.3337443470954895, Validation Loss: 344.7869, Test Loss: 343.7960
Epoch 5/200, Train Loss: 342.0004, kl :0.2716023325920105, Validation Loss: 342.5914, Test Loss: 341.4882
Epoch 6/200, Train Loss: 340.2277, kl :0.24269886314868927, Validation Loss: 341.7125, Test Loss: 340.5168
Epoch 7/200, Train Loss: 339.8735, kl :0.25246214866638184, Validation Loss: 379.4231, Test Loss: 379.0664
Epoch 8/200, Train Loss: 339.1463, kl :0.2320970743894577, Validation Loss: 341.4349, Test Loss: 340.2191
Epoch 9/200, Train Loss: 339.6302, kl :0.20704437792301178, Validation Loss: 341.2012, Test Loss: 339.9948
Epoch 10/200, Train Loss: 339.3866, kl :0.2

Test Loss,▃▃▃▃▃▁▁▃▁▅▁▁▁▅▁▁▁▁▁▅▁▁▅▆▅▁▁▁▄▁▅▁▁█▁▆▁▆▁▆
Test anmaly,███▆▆▅▇▇▆▅▆▆▄▄▅▃▃▃▃▅▃▃▄▃▅▂▂▂▄▃▃▄▂▅▁▁▃▁▁▁
Test kl,▁▁▁▃▃▁▁▁▁▃▃▁▄▅▁▁▄▁▅▄▅▁▁▄▁▁▁▅▅▁▁▁▁▁▁▄▁▁█▅
Train Loss,█▆▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Loss,▂▃▁▃▂▃▃▁▁▂▅▁▁▄▁▄▁▁▁▁▁▁▅▅▅▁▁▁▁▅█▆▇▁██▁▇█▁
anmaly,█▆▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▁▂▂▂▂▂▁▁▁▁
kl,▁▁▃▃▃▃▃▁▁▁▆▃▅▁▁▁▁▅▁▁▁█▂▁▅▁▁▁▆▆▁▁▅█▇▁▁▂▁▁
Test Loss,339.14624
Test anmaly,0.01689
Test kl,4.63207
Train Loss,334.21554
